#### Import necessary library

In [51]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras import layers
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout


In [52]:
train_dir = 'C:\\Users\\alan\\AI Project\\dataset\\aclImdb\\train'
val_dir = 'C:\\Users\\alan\\AI Project\\dataset\\aclImdb\\test'
batch_size = 32

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    class_names=['neg', 'pos'])

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    val_dir,
    batch_size=batch_size,
    class_names=['neg', 'pos'])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [53]:
# Define the text vectorization layer
vectorize_layer = layers.TextVectorization(
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=500)

In [54]:
# Fit the text vectorization layer to the training data
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [55]:
# Define a function to apply the text vectorization layer to the data
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [56]:
# Apply the text vectorization layer to the training and validation data
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [57]:
model = tf.keras.Sequential([
    layers.Embedding(input_dim=10000, output_dim=16),
    GRU(16),
    layers.Dense(1, activation='sigmoid')
])


In [58]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10)